<a href="https://colab.research.google.com/github/GeoDust/Chihuahuan/blob/main/Chihuahuan_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#A Look at the Desertification of the Chihuahuan Desert Using Open Source Data
##Principle Component Analysis 
Part one of the PCA to get it going with one image before getting the images over time 

##Uploads and Installs
###Python installations for running Scripts
*   Miscellaneous operating system interfaces https://docs.python.org/3/library/os.html
*   Earth Engine https://developers.google.com/earth-engine/apidocs
*    Pandas https://pandas.pydata.org
*   Geopandas https://geopandas.org/docs/reference.html
*   Numpy https://numpy.org/doc/stable/reference/index.html
*   Matplotlib.pyplot https://matplotlib.org/stable/api/index.html#the-pyplot-api
*   Matplotlib.dates https://matplotlib.org/stable/api/dates_api.html
*   Google Colab
*   Google Drive



In [ ]:
!pip install geopandas
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap']) 
#import os
import ee
import geemap 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Paths and rows that touch the Schmidt outline

In [ ]:
AOIPathsRows = ee.FeatureCollection('users/marcosmendezlordsburg/Chihuahuan/landsatPathRow')

##Row and Path information 
This is temporary 

In [ ]:
# PathCurrent and RowCurrent are used for testing. They will change later so that each tile can be evaluated
PathCurrent = ee.Number(33)
RowCurrent = ee.Number(38)

datasetCurrent = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
            .filter(ee.Filter.eq('WRS_ROW',RowCurrent))
            .filter(ee.Filter.eq('WRS_PATH',PathCurrent))
            .filter(ee.Filter.eq('CLOUD_COVER',0))
            .select('SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7'))

##Getting area of interest
Collection 2 images are re-baselined. This means that images are repositioned to more closely match with ground control points in an effort to line up with images from Sentinel-2 for improved ground acuracy.

Because of the re-baselining, an AOI will have to be created so that all images will calculate in the same area.

Here is how to get the AOI
* Get the geometry around each image 
* Intersect all image geometries to find area where all images have information.  
* Repeat for each tile

To get the image geometry interesections, we use:
* ee.ImageCollection.geometry
** Gets the geometries for all images in the image collection
* ee.Geometry.intersection 
** Gets the intersection of two image geometries
* ee.FeatureCollection.iterate
** a type of looping that will repeat intersections for the image collection geometires





In [ ]:
#accumulator takes the intersected geometry for all images in the tile
def accumulator(current, previous):
    previous = ee.Geometry(ee.List(previous))
    nextOne = ee.Geometry(current)
    Updated = ee.Geometry(nextOne.intersection(previous))
    return ee.List(Updated)

#ROICollection takes the paths and rows to make a feature from the geometry
def ROICollection(path,row):
#    pathRow = ee.Feature(pathsRows)
    imgPath = path
    imgRow = row
    imgCollection = (ee.ImageCollection(datasetCurrent)
                    .filterDate('1980-01-01', '2020-12-31')
                    .filter(ee.Filter.eq('WRS_PATH', imgPath))
                    .filter(ee.Filter.eq('WRS_ROW', imgRow)))
    imgColGeometires = ee.List(ee.Geometry(imgCollection.geometry()).geometries())
    initial = ee.Geometry(imgColGeometires.get(0))
    intersectedGeometry = ee.Geometry(ee.List(imgColGeometires.iterate(accumulator,initial)))
    areaProperties = {'Path':imgPath,'Row':imgRow}
    return ee.Feature(intersectedGeometry,areaProperties)

#This is the AOI for this particular tile. the fucntion ROICollection can be mapped.
region = (ROICollection(PathCurrent,RowCurrent))

In [ ]:
region.getInfo()

In [ ]:
#trying to see one image for trial
image = ee.Image(datasetCurrent.first().clip(region))

In [ ]:
#Scale of 30 meters is known as the size of each pixel on landsat. If another dataset is used, scale can be changed to a variable controled by the image properties.
scale = ee.Number(30)
bandNames = image.bandNames();

In [ ]:
#Mean center the data to enable a faster covariance reducer and an SD stretch of the principal components.

meanDict = image.reduceRegion(**{
    'reducer' : ee.Reducer.mean(),
    'geometry' : region.geometry(),
    'scale' : (scale),
    'maxPixels' : (1e9)
});
means = ee.Image.constant(meanDict.values(bandNames));
centered = image.subtract(means);
#centered = image

In [ ]:
# This helper function returns a list of new band names.
def getNewBandNames(prefix):
  seq = ee.List.sequence(1, bandNames.length())
  return seq.map(lambda b: ee.String(prefix).cat((ee.Number(b).int()).format()))


#getting the PCA

In [ ]:
def getPrincipalComponents(centered, scale, region):
  # Collapse the bands of the image into a 1D array per pixel.
  arrays = centered.toArray()

  # Compute the covariance of the bands within the region.
  covar = arrays.reduceRegion(**{
     'reducer': ee.Reducer.centeredCovariance(),
     'geometry': region.geometry().getInfo(),
     'scale': scale,
     'maxPixels': 1e9
  })
    # Get the 'array' covariance result and cast to an array.
    # This represents the band-to-band covariance within the region.
  covarArray = ee.Array(covar.get('array'))


  # Perform an eigen analysis and slice apart the values and vectors.
  eigens = covarArray.eigen()

  # This is a P-length vector of Eigenvalues.
  eigenValues = eigens.slice(1, 0, 1)
  # This is a PxP matrix with eigenvectors in rows.
  eigenVectors = eigens.slice(1, 1)

  # Convert the array image to 2D arrays for matrix computations.
  arrayImage = arrays.toArray(1)

  # Left multiply the image array by the matrix of eigenvectors.
  principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

  # Turn the square roots of the Eigenvalues into a P-band image.
  sdImage = ee.Image(eigenValues.sqrt()).arrayProject([0]).arrayFlatten([getNewBandNames('sd')])

  # Turn the PCs into a P-band image, normalized by SD.
  return (principalComponents
          .arrayProject([0])
          .arrayFlatten([getNewBandNames('PC_')])
          .divide(sdImage))


In [ ]:
pcImage = getPrincipalComponents(centered, scale, region)

In [ ]:
#Visualization Parameters

img_vis_params_rbg = {
    'bands': ['SR_B3','SR_B2','SR_B1'],
    'min' : 0,
    'max' : 20000,
}

img_vis_params_pca = {
    'bands': ['PC_1', 'PC_2', 'PC_3'],
    #'min' : -10,
    #'max' : 10,
}

In [ ]:
pcImageMinMax = pcImage.reduceRegion(**{
    'reducer':ee.Reducer.minMax(),
    'geometry':region.geometry(),
    'scale': scale,
    'maxPixels': 1e18
    })

In [ ]:
pcImageMinMax.getInfo()
#pcImageMinMax

In [ ]:
bandNamesList = pcImage.bandNames().getInfo()

Map = geemap.Map()
for band0 in bandNamesList:
    band = pcImage.select(band0)
    Map.addLayer(band,{min:2,max:2},band0)
Map.addLayer(image,img_vis_params_rbg,"RGB Image")
Map.addLayer(pcImage,img_vis_params_pca,'PCA IMAGE')
Map.center_object(pcImage)
Map

In [ ]:
import geemap.chart as chart

In [ ]:
sampleSettings = {
    'region': region.geometry().getInfo(),
    'scale': scale,
    'numPixels': 1e9,

}
sample1 = pcImage.sample(**sampleSettings)

In [ ]:
sample1.getInfo()